In [1]:
import xarray as xr
import xskillscore as xs
from dask.distributed import Client
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
import sys

In [2]:
vels = np.arange(0,30,1)
power_dtu = [0, 0, 0, 0, 280.2, 799.1, 1532.7, 2506.1, 3730.7, 5311.8, 
             7286.5, 9698.3, 10640, 10640, 10640, 10640, 10640, 10640, 10640, 10640, 
             10640,  10640,  10640, 10640, 10640, 10640, 0, 0,0,0]

def p_curve_gufunc(ws):
    f_curve = interpolate.interp1d(vels,power_dtu, bounds_error=False, fill_value=0)
    return f_curve(ws)

def p_curve(ws):
    return xr.apply_ufunc(p_curve_gufunc, ws, dask="parallelized", output_dtypes=[float])

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 56,Total memory: 62.44 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38327,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 56
Started: Just now,Total memory: 62.44 GiB
Comm: tcp://127.0.0.1:40501,Total threads: 7
Dashboard: http://127.0.0.1:41419/status,Memory: 7.80 GiB
Nanny: tcp://127.0.0.1:43453,


2024-07-23 11:43:50,125 - distributed.protocol.core - CRITICAL - Failed to Serialize
Traceback (most recent call last):
  File "/apps/anaconda3/envs/basico/lib/python3.10/site-packages/distributed/protocol/core.py", line 109, in dumps
    frames[0] = msgpack.dumps(msg, default=_encode_default, use_bin_type=True)
  File "/apps/anaconda3/envs/basico/lib/python3.10/site-packages/msgpack/__init__.py", line 36, in packb
    return Packer(**kwargs).pack(o)
  File "msgpack/_packer.pyx", line 294, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 300, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 297, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 264, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 231, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 264, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 272, in msgpack._cmsgpack.Packer._pack
ValueError: memoryview is too 

In [4]:
ls 

'DataArray_to_Numpy_SCC_Historical (1).py'   spearman_calc-Copy1.ipynb
'DataArray_to_Numpy_spearman_all (1).py'     spearman_calc.ipynb
 Untitled.ipynb                             'spearman_dask (1) (1).ipynb'
 log_mex_spearman.txt                        spearman_dask.ipynb
 scripts/                                    spearman_mexico_dask.ipynb
 spearman_by_season_mex.py


In [5]:
#data = xr.open_mfdataset("NAS-eolica/3_data/ERA5/latam/ERA5_latam_201*.nc", data_vars=["u100","v100"],
#                         chunks={'latitude':5, "longitude":5})
data = xr.open_mfdataset("../eolica-nas/3_data/ERA5/mexico/ERA5_mexico_201*.nc",
                         chunks={'latitude':50, "longitude":50})
# data = xr.open_mfdataset("NAS-eolica/3_data/ERA5/mexico/ERA5_mexico_201*.nc")
data["temp_module"] = data["t2m"] + ((data["ssrd"] /3600) / 1000 * (320.15 - 293.15) / 800)
data["power_solar"] = data["temp_module"] - 298.15
data["power_solar"] = (data["power_solar"] * -0.005 + 1) * 0.93 * 190
data["power_solar"] = data["power_solar"] * (data["ssrd"] /3600) / 1000
data["cf_sol"] = data["power_solar"]/190
data["ws100"] = np.sqrt(data.u100**2+data.v100**2)
# data["cf_viento"] = f_curve(data.ws100)/10640

In [6]:
data

<xarray.Dataset> Size: 146GB
Dimensions:      (time: 87648, latitude: 85, longitude: 245)
Coordinates:
  * longitude    (longitude) float32 980B -120.0 -119.8 -119.5 ... -59.25 -59.0
  * latitude     (latitude) float32 340B 34.0 33.75 33.5 ... 13.5 13.25 13.0
  * time         (time) datetime64[ns] 701kB 2010-01-01 ... 2019-12-31T23:00:00
Data variables:
    u100         (time, latitude, longitude) float64 15GB dask.array<chunksize=(8760, 50, 50), meta=np.ndarray>
    v100         (time, latitude, longitude) float64 15GB dask.array<chunksize=(8760, 50, 50), meta=np.ndarray>
    u10          (time, latitude, longitude) float64 15GB dask.array<chunksize=(8760, 50, 50), meta=np.ndarray>
    v10          (time, latitude, longitude) float64 15GB dask.array<chunksize=(8760, 50, 50), meta=np.ndarray>
    t2m          (time, latitude, longitude) float64 15GB dask.array<chunksize=(8760, 50, 50), meta=np.ndarray>
    ssrd         (time, latitude, longitude) float64 15GB dask.array<chunksize=(8760, 50, 50), meta=np.ndarray>
    temp_module  (time, latitude, longitude) float64 15GB dask.array<chunksize=(8760, 50, 50), meta=np.ndarray>
    power_solar  (time, latitude, longitude) float64 15GB dask.array<chunksize=(8760, 50, 50), meta=np.ndarray>
    cf_sol       (time, latitude, longitude) float64 15GB dask.array<chunksize=(8760, 50, 50), meta=np.ndarray>
    ws100        (time, latitude, longitude) float64 15GB dask.array<chunksize=(8760, 50, 50), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-03-27 15:40:10 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [7]:
%%time
data["cf_wind"] = p_curve(data.ws100).compute()

CPU times: user 24.1 s, sys: 17.1 s, total: 41.2 s
Wall time: 48.6 s


In [8]:
data["cf_wind"] = data["cf_wind"]/10640

In [17]:
cfs = data[["cf_wind", "cf_sol"]].copy()

In [18]:
cfs.to_netcdf("cfs_mexico.nc")